In [1]:
import numpy as np
def paa(x, w):
    #w is the number of segments
    #first calculate the length of the time series
    n = len(x)
    #now calculate the length of each segment
    step = n/w
    #now calculate the paa
    paa = []
    for i in range(w):
        paa.append(np.mean(x[int(i*step):int((i+1)*step)]))
    return paa
    

In [58]:
from sklearn.neighbors import KDTree

X= np.array([[1, 1,1,1], [2, 2,2,2], [3, 3,3,3]])
tree = KDTree(X, leaf_size=2) 
dist, ind = tree.query([[4,4,4,4]], k=3)        
print(ind)  # indices of 3 closest neighbors

[[2 1 0]]


In [33]:
import os
from rtree import index
p = index.Property(dimension=4)
ucr_idx = index.Index(property=p)


counter=0
segments =[]
for filename in os.listdir("./test_dataset/UCR"):
    f =open("./test_dataset/UCR/" + filename + "/" + filename+"_TEST")
    content = f.read()
    content = content.split("\n")
    content = content[0]
    content = content.split(",")
    content = content[1:]
    content = list(map(float, content))
    paa_4 = paa(content, 4)
    #ucr_idx.insert(counter, (paa_4[0], paa_4[1], paa_4[2], paa_4[3]),obj=filename)
    segments.append(paa_4)
    counter+=1
    
    f.close()

print(segments)
tree1 = KDTree(segments, leaf_size=2,metric='l2')




[[-0.461969552238806, -0.76471, 1.1801809850746265, 0.05706313235294119], [-0.24725825000000004, 0.3130921818181818, -0.34955934090909097, 0.2837404090909091], [0.16850219354838705, -0.13239509523809523, -0.1953653492063492, 0.16192333333333334], [0.36032730341880337, 0.02708088305084746, 0.9155792256410256, -1.2921731440677968], [0.3909733671875, -0.4626831484374999, 0.548116765625, -0.47640924218749997], [0.02241166953125001, -0.49304124609375, 0.6490567578124999, -0.17842823437499994], [-0.20108214236111113, 0.4216050625, -0.5483521326388889, 0.32557117241379313], [-0.34735034374999996, 1.247591, -0.45301916875000003, -0.44722590624999997], [0.6034848536585368, -0.2636030476190476, -0.23779721951219512, -0.0933704761904762], [0.5945058924205379, -0.41260005512195125, -0.33271322858536584, 0.1522578184634146], [-0.49381297605633817, 0.5692430555555554, 0.6104339000000002, -0.6842403472222222], [-0.5962179999999999, 0.25842977777777787, 0.9436506666666667, -0.6058819999999999], [0.071

In [41]:
#now we have the index, we can search for the nearest neighbor
#first we need to create the query
f = open("./test_dataset/UCR/Adiac/Adiac_TEST")       
content = f.read()
content = content.split("\n")
content = content[4]
content = content.split(",")
content = content[1:]
content = list(map(float, content))
paa_4 = paa(content, 4)
f.close()

dist, ind = tree1.query([paa_4], k=50)
print(len(ind[0]))  # indices of 3 closest neighbors
print(dist)  # distances to 3 closest neighbors    

# #now we can search for the nearest neighbor
# res = es.search(index="paa", 
#                 body={"knn":{"field": "paa","query_vector": paa_4,"k": 4,"num_candidates": 10},
#                       "fields":["label"]})

# for hit in res["hits"]["hits"]:
#     print(hit["_source"]["label"]+" "+str(hit["_score"]))

50
[[0.12433397 0.14390158 0.21001938 0.23943045 0.29465042 0.3057488
  0.32366134 0.34967826 0.35077368 0.3553244  0.41740888 0.43934262
  0.48645236 0.59745352 0.69868245 0.74794486 0.74865941 0.75340884
  0.76348432 0.8380186  0.9118388  0.95826323 1.01935695 1.02156938
  1.0266748  1.10511633 1.11187113 1.12990499 1.13354356 1.18716888
  1.24375253 1.24375253 1.25808982 1.29729321 1.34090137 1.35515354
  1.35515354 1.357251   1.39128003 1.40958185 1.41172375 1.44603668
  1.46216495 1.47254766 1.4725523  1.50938034 1.59430865 1.63297734
  1.78058188 1.90851764]]


In [48]:
mappings ={
    "mappings": {
        "properties": {
            "midi_paa": {
                "type": "dense_vector",
                "dims": 16,
                "similarity": "l2_norm"
            },
            "label": {
                "type": "keyword"
            }
        }
    }
}

['euclidean', 'l2', 'minkowski', 'p', 'manhattan', 'cityblock', 'l1', 'chebyshev', 'infinity']


In [64]:
import os 
import pretty_midi

# p = index.Property(dimension=4)
# midi_idx = index.Index(property=p)

index = []
files = {}
counter = 0
for filename in os.listdir("./test_dataset/dataset_beetles/"):
    try:
        midi_data = pretty_midi.PrettyMIDI("test_dataset/dataset_beetles/" + filename)
    except:
        continue
    song_duration = midi_data.get_end_time()
    segment_duration = 15
    tempo = midi_data.estimate_tempo()
 
    
    for i in range(0, int(song_duration) - segment_duration):
        prev = 1
        segment = []
        for j in range(prev, len(midi_data.instruments[0].notes)):
            note = midi_data.instruments[0].notes[j]
            if note.start > i and note.end < i + segment_duration:
                #segment.append(note.pitch - midi_data.instruments[0].notes[j - 1].pitch)
                segment.append(round(note.pitch,2))
        prev = j + 1
        segment = paa(segment, 16)
        #if segment contiain nan, skip it
        if np.isnan(segment).any():
            continue
        counter += 1
        index.append(segment)
        # try:
        #     midi_idx.insert(counter, (segment_4[0], segment_4[1], segment_4[2], segment_4[3]),obj=filename)
        #     counter+=1
        # except:
        #     continue
    files[filename] = counter
    f.close()

midi_tree = KDTree(index, leaf_size=2,metric='l2')
print(files)
        

C:\Users\SP Singh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\SP Singh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\SP Singh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


{'The_Beatles_-_All_My_Loving.mid': 99, 'The_Beatles_-_All_You_Need_Is_Love.mid': 265, 'The_Beatles_-_A_Day_in_the_Life.mid': 392, "The_Beatles_-_A_Hard_Day's_Night.mid": 526, "The_Beatles_-_Can't_Buy_Me_Love.mid": 625, 'The_Beatles_-_Come_Together.mid': 886, 'The_Beatles_-_Day_Tripper.mid': 1051, 'The_Beatles_-_Drive_My_Car.mid': 1207, 'The_Beatles_-_Eight_Days_A_Week.mid': 1358, 'The_Beatles_-_Eleanor_Rigby.mid': 1358, 'The_Beatles_-_From_Me_to_You.mid': 1458, 'The_Beatles_-_Getting_Better.mid': 1590, 'The_Beatles_-_Get_Back.mid': 1717, 'The_Beatles_-_Got_to_Get_You_Into_My_Life.mid': 1839, 'The_Beatles_-_Hello_Goodbye.mid': 2012, 'The_Beatles_-_HELP!.mid': 2129, 'The_Beatles_-_Helter_Skelter.mid': 2283, 'The_Beatles_-_Here_Comes_the_Sun.mid': 2393, 'The_Beatles_-_Hey_Jude.mid': 2601, 'The_Beatles_-_In_My_Life.mid': 2733, 'The_Beatles_-_I_Am_the_Walrus.mid': 2839, 'The_Beatles_-_I_Feel_Fine.mid': 2957, 'The_Beatles_-_I_Saw_Her_Standing_There.mid': 3099, 'The_Beatles_-_I_Want_to_Hold_

In [66]:
import feature_extraction.extract as extract
absolute_pitch, relative_pitch, tempo = extract.extract("user_input/query.wav", apitch=True, tempo=True, rpitch=True)
paa_4 = paa(relative_pitch, 16)
dist, ind = midi_tree.query([paa_4], k=100)
print(ind)
print(dist)


Predicting MIDI for user_input/query.wav...
[[4468 4620 4534 4469 4621 4533 4467 4532 4535 4622 4470 4619 4536 4626
  4623 4538 4473 4625 4557 4472 4537 4474 4539 4558 4624 4466 4618 4475
  4627 4471 4556 4540 4559 4561 4541 2676 2628 4631 4543 4478 4630 4476
  4629 4542 4477 2714 2716 4544 4479 2677 2629 4628 4560 4555 2715 4487
  4553 4617 4531 4545 4552 4486 4554 4480 4632 2627 2675 4546 2713 4562
  2647 4638 2678 2630 4481 4563 4488 4639 4633 4483 2626 2674 4637 4485
  4547 4482 2722 4548 4551 4640 2717 4635 4465 4530 2679 2631 4489 4634
  2683 2635]]
[[123.02641993 123.02641993 123.91327612 123.91327612 123.93143266
  124.23163848 124.54818345 124.54818345 124.77279351 124.78281132
  124.78281132 125.13392825 127.97460686 128.55187021 128.56807535
  128.62639698 129.13849155 129.13849155 129.19462063 129.2091328
  129.2091328  129.2211713  129.2211713  129.25190736 129.29037087
  129.44400334 129.44400334 129.44883159 129.44883159 129.49903475
  129.55382837 129.75596754 130.02136